# Notes on using MoT-Voellmy

## General features

MoT-Voellmy is NGI's model for calculating the run-out of snow avalanches. It is a *continuum model*, i.e., the avalanche is treated as a (fluid) body rather than as just a point mass as in the block models. Thus, the length, width and thickness of the flow are variable both in space and time. The dynamical equations governing the flow are derived from the requirement that mass and momentum must be conserved (if erosion and deposition along the path and the external forces of gravity and friction are properly accounted for). All of this will be discussed in more detail during one of the lectures.

The "trick" behind numerical continuum models is to discretize space and time into sufficiently small finite time steps, line segments (in 1D), subareas (in 2D) or elementary volumes (in 3D). This procedure reduces the number of degrees of freedom from infinite to a finite number, so that the fundamental equations no longer are partial differential equations but (usually highly non-linear) algebraic equations. These can be solved numerically on a computer.

In many respects, MoT-Voellmy is similar to the widely used (commercial) code [RAMMS::AVALANCHE](./Christen_et_al.RAMMS.CRST_63_2010.pdf), but its numerics are simpler, which makes it very fast. Both programs do not solve the basic equations for the 3D velocity vector $(u, v, w)$ in 3D, but instead assume the flow to always follow the terrain surface. This makes it possible to reduce the equations to 2D equations for the slope-parallel *depth-averaged* velocity $(\bar{u}, \bar{v})$ and the flow depth (or flow thickness) $h$, which is measured perpendicular to the terrain. Thus, one loses all information about the variability of $u$ and $v$ in the direction normal to the terrain. On the upside, the computational effort is reduced by some two orders-of-magnitude. Knowledge of the depth-averaged velocity is sufficient in most practical applications, like hazard mapping or dimensioning of mitigation measures.

## Required input data

MoT-Voellmy is started from the command line with one argument, the name of the *run configuration file* (RCF) as

`<path to the code>/MoT-Voellmy.2022-10-29 <RCF>`

on Linux and macOS, or as

`<path to the code>\MoT-Voellmy.2022-10-29.exe <RCF>`

on MS Windows.

In the RCF, the user specifies the spatial input data by listing the corresponding raster files and the model parameters. In the folder for MoT-Voellmy, there is a template RCF. All the data can be changed in a text editor, but all lines must be present and their sequence preserved (including comment lines starting with \#). The keys on the left side must not be changed. The RCF should be renamed so that the user will still remember the purpose of the simulation some years later. It is recommended to save the RCFs in the folder or subfolder of the current project.

**Auxiliary information:**  
General information on the present run, not relevant for the program but for the user:  
`MoT-Voellmy input file version          2021-10-25`   (do not change!)  
`Area of Interest                        11192_Volda`  
`UTM zone                                33N`  
`EPSG geodetic datum code                25833`  
`Run name                                RA6157_df2_v26_none`  

**Flags:**  
These parameters take the values `yes` or `no`, except for the flag `Entrainment` with options `none`, `RAMMS`, `TJEM`, `AvaFrame` or `GOEM`, the flag `Rheology`, which is presently limited to `Voellmy`, and the flag `Parameters` with the alternatives `constant` and `variable`. With `constant`, spatially constant values of $\mu$ and $k$ are assumed (see below under *Physical parameters* and *Raster input data*). 

`Centrifugal effects                     yes`  
If "yes". modifies the terrain-normal component of gravity as $g \cos\theta \longrightarrow g \cos\theta + \kappa (u^2+v^2)$.

`Forest effects                          yes`  
Enables or disables braking of the flow by the forest and breaking of the forest by the flow.

`Entrainment                             none`  
Selects no bed entrainment or one of three entrainment models. The options "RAMMS" or "AvaFrame" may produce unrealistic results.

`Deposition                              no`  
This option is not implemented in the present version of MoT-Voellmy.

`Evolving geometry                       no`  
If entrainment is enabled, the change of the terrain due to erosion is tracked and may induce self-channeling of the flow.

`Write velocity vectors                  no`  
If enabled, the code writes not only the modulus of the velocity, $\sqrt{\bar{u}^2 + \bar{v}^2}$ but also its components $\bar{u}$, $\bar{v}$ to separate files.

`Write maximum pressure                  no`  
If enabled, the maximum impact pressure $p_{\mathrm{imp}} = \rho (\bar{u}^2 + \bar{v}^2)$ in each cell over the entire simulation is written to file.  
`Write instant. pressure                 no`  
If enabled, the instantaneous impact pressure $p_{\mathrm{imp}} = \rho (\bar{u}^2 + \bar{v}^2)$ in each cell is written to file whenever output is generated.

**Physical parameters:**  
`Gravitational acceleration   (m/s^2)    9.81`  
This value can be changed to simulate extra-terrestrial or sub-marine landslides.

`Density                     (kg/m^3)    250.0`  
The flow density matters only for the impact pressure because the friction force is assumed proportional to $\rho$.

`Dry-friction coefficient         (-)    0.35`  
`Turbulent drag coefficient       (-)    0.0023`  
If the flag `Parameters` is set to `constant `, the values for $\mu$ and $k$ must be set here. Else, raster file names containing the spatially variable values are to be specified instead.

`Effective drag height            (m)    0.0`  
This parameter modifies the flow-depth dependence of the drag term. If set to a very small value $0 < h_{\mathrm{eff.}} \ll 1\,\mathrm{m}$, the model behaves like the PCM model.

`Passive earth-pressure coeff.    (-)    1.0`  
Values < 1.0 can be used to stabilize the simulation in very rugged terrain by reducing the hydrostatic pressure between cells. Keep it at 1.0 whenever possible.

**Numerical parameters**

`Output filename root                    Run_26`  
All generated output files will be given this prefix so different runs can be distinguished. It may also be of the form `<subfolder>/<some prefix>` to place the files in a separate subfolder.

`Output format                           ESRI_ASCII_Grid`  
The output raster files can be written either in ESRI ASCII Grid format (bulky but human readable) or in `BinaryTerrain` (version 1.3) format (faster and more compact). Both formats are understood by QGIS.

`Simulation time                  (s)    180.0`  
This value specifies after which simulated time the computation is stopped, even though some parts of the flow (typically in the tail) may sdtill be moving.

`Minimum time step                (s)    0.0001`  
`Maximum time step                (s)    0.5000`  
`Initial CFL number               (-)    0.8`  
In the time discretization, MoT-Voellmy uses variable timesteps to balance numerical stability (short timesteps) against efficiency (long timesteps). The timestep must be shorter the time it takes a wave to traverse a computational cell. The CFL number specifies how close to this theoretical limit the time step is allowed to be; this number must be < 1.0. The code keeps it, however, between the minimum and maximum time step chosen by the user. If the time step chosen by the code falls below the minimum value, the simulation has diverged and is stopped.

`Output interval                  (s)    1.0`  
Intermediate results are written to files at fixed intervals. At the end of the simulation, the maximum values, over the entire simulated interval, of $\bar{u}$, $\bar{v}$, $h$ and optionally of some other variables are written to files as well.

`Minimum flow depth               (m)    0.05`  
`Minimum speed                  (m/s)    0.1`  
`Momentum threshold          (kg m/s)    100.0`  
Cells in which the flow depth, the speed or the momentum fall below the specified values will be considered stopped. They may, however, be remobilized if mass from neighboring cells pushes them strongly enough to overcome friction.

**Raster input data:**

All the raster input data must be in ESRI ASCII Grid format and cover the same rectangular area at the same resolution. This means that soem clipping and interpolation may be required when preparing the data for a simulation.
- *Digital terrain model* (DTM) over a rectangular area with a spatial resolution of typically 5–10 m (for snow avalanches). If the resolution is poorer, important terrain features are not properly resolved. If the resolution is much finer, the flow moves over terrain that is much more hummocky than the snow cover in wintertime. The grid must be regular, i.e., consist of square cells when projected onto a horizontal plane.  
`Grid filename                           RA6157_dem.asc`
- *Release area and fracture depth* are specified by the value 0 in non-release-area cells and the slab depth (in m, measured normal to the terrain) in the release area. There may be several disconnected release areas if one assumes they are released simultaneously. The fracture depth may vary inside a release area.  
`Release depth filename                  RA6157_wf_h.asc`

Depending on the desired simulation, several further raster files may need to be specified:
- *Friction parameters $\mu$ and $k$*: If the friction parameters are assumed to vary spatially variable (reflecting spatial variation of snow properties), they must be specified in two raster files. The dimensionless parameter $k$ relates to the traditional Voellmy parameter $\xi$ with units m/s² as $k = g/\xi$.  
`Dry-friction coefficient         (-)    RA6157_mu.asc`  
`Turbulent drag coefficient       (-)    RA6157_k.asc`
In addition, set
`Parameters                              variable`  
- *Start velocity*: To start a simulation with the flow mass already in motion, one can specify the velocity components $\bar{u}$ and $\bar{v}$ (in m/s) in two raster files. If starting from rest, use hyphens instead of file names.
`Start velocity u filename               -`
`Start velocity v filename               -`
- If *entrainment of bed material* shall be simulated, two raster files specifying the local initial depth $b_0$ (in m) and shear strength $\tau_c$ (in Pa) of the erodible material must be supplied.  
`Bed depth filename                      RA6157_b0.asc`  
`Bed shear strength filename             RA6157_tauc.asc`
- *Interaction of the flow with forest* requires two raster files giving the local values of $n\cdot D$ and $D$, where $n$ in 1/m² is the local tree density and $D$ (in m) is the local average tree diameter. In addition a global value for the *modulus of rupture* (MoR, in MPa, default 50 MPa) characterizing the average rupture strength of the wood must be set in the RCF. Set this parameter to a very high value if you wish to disable breaking or uprooting of the trees.  
`Forest density filename                 RA6157_nD.asc`  
`Tree diameter filename                  RA6157_tD.asc`

---

Below, the file snippets shown above are assembled into a complete RCF:
```
# Run information
#
MoT-Voellmy input file version          2021-10-25
Area of Interest                        11192_Volda
UTM zone                                33N
EPSG geodetic datum code                25833
Run name                                RA6157_df2_v26_none
#
# File names
#
Grid filename                           RA6157_dem.asc
Release depth filename                  RA6157_wf_h.asc
Bed depth filename                      RA6157_b0.asc
Bed shear strength filename             RA6157_tauc.asc
Forest density filename                 RA6157_nD.asc
Tree diameter filename                  RA6157_tD.asc
Start velocity u filename               -
Start velocity v filename               -
Output filename root                    Run_26_none
Output format                           ESRI_ASCII_Grid
#
# Physical parameters
#
Gravitational acceleration   (m/s^2)    9.81
Density                     (kg/m^3)    250.0
Rheology                                Voellmy
Parameters                              variable
Dry-friction coefficient         (-)    RA6157_mu.asc
Turbulent drag coefficient       (-)    RA6157_k.asc
Effective drag height            (m)    0.0
Centrifugal effects                     yes
Passive earth-pressure coeff.    (-)    1.0
#
Forest effects                          yes
Tree drag coefficient            (-)    1.0
Modulus of rupture             (MPa)    50.0
Forest decay coefficient       (m s)    0.15
#
Entrainment                             none
Erosion coefficient              (-)    0.0
Bed strength profile                    global
Bed friction coefficient         (-)    0.25
Bed density                 (kg/m^3)    140.0
Deposition                              no
Evolving geometry                       no
#
# Numerical parameters
#
Simulation time                  (s)    180.0
Minimum time step                (s)    0.0001
Maximum time step                (s)    0.5000
Output interval                  (s)    1000.0
Write velocity vectors                  no
Write maximum pressure                  no
Write instant. pressure                 no
Minimum flow depth               (m)    0.05
Minimum speed                  (m/s)    0.1
Momentum threshold          (kg m/s)    100.0
Initial CFL number               (-)    0.8
```

---

## Visualization of results

MoT-Voellmy prints information on the progress of the simulation to the screen (unless it is running within NAKSIN 4). This allows the user to judge whether there are any irregularities that require closer scrutiny.

The simulation results are output as raster files:
- The file format can be chosen in the run configuration file as `ESRI_ASCII_Grid` or `BinaryTerrain 1.3`. Both formats can be read by QGIS. BinaryTerrain 1.3 is read and written somewhat faster and the files are more compact, but ESRI ASCII Grid fies can usually be compressed better than BinaryTerrain 1.3 files.
- Two types of files can be written to disk:
    * *Time slices* showing the values of one field variable at a given time, e.g., $h(x_i, y_j, t_*)$, the flow depth at time $t_*$ in every grid cell $(i,j)$
    * *Maximum values* of one field variable over the entire simulation, i.e., $\max_{t_n} h(x_i,y_j, t_n)$ at each cell $(i,j)$
- The initial conditions $h(x_i, y_j, t_0)$ and $s(x_i, y_j, t_0)$ and the maximum fields $h_{\mathrm{max}}(x_i, y_j)$ and $s_{\mathrm{max}}(x_i,y_j)$ are always written. $s = \sqrt{u^2 + v^2}$ is the speed or celerity.
- In addition, the user can which other variables among the velocity components $u$, $v$, the pressure $p$, the erodible snow-cover depth $b$ and the forest stand density $n$ should be output.
- Time slices are written at times $k \Delta t_{\mathrm{out}}$, where $\Delta t$ is the output interval in seconds. If this interval is smaller than the minimum timestep, time slices are written after each timestep. If $\Delta_{mathrm{out}} is larger than the simulation time, only the initial, final and maximum values are written to file.

How the results are visualized, will depend on the purpose of the simulation and personal preferences. Often it is helpful to load at least a topographic background map (if available), the DTM, the initial condition $h(x_i, y_j, t_0)$ and the maxima of $h$ and $s$. If no background map is available, contour lines can be created in QGIS from the DTM. (Raster – Extract – Contour, typically thin 10 m contour lines and thicker 100 m contour lines help the user grasp the terrain properties). To faciitate analysis of the results, suitable choices for the color palettes and the values of layer transparency should be found and saved as styles.

---

To test whether MoT-Voellmy works correctly in your set-up, find the cell below that corresponds to your operating system, edit the path of the MoT-Voellmy executable, and run the cell. It may take a while (around a minute on some systems) before anything happens because the progress messages from MoT-Voellmy are not written to the screen before the end of the simulation if running the program within the Jupyter notebook.

Alternatively, you can run the simulation from the command line of a terminal window. This will show the progress and may be faster.

In MS Windows, use the following cell:

In [ ]:
! P:\\2021\\04\\20210451\\PD2_Japan_2023\\Online_module_2\\02_Programs\\02.6_Using_NAKSIN_4\\NAKSIN_4\\MoT-Voellmy-win64.2022-10-30.exe Test_case_MoT-Voellmy/Ryggfonn_1987-01-28_cst_noent.rcf

With macOS, use this line:

In [ ]:
! ../02.6_Using_NAKSIN_4/NAKSIN_4/MoT-Voellmy-macOS.2022-10-30 Test_case_MoT-Voellmy/Ryggfonn_1987-01-28_cst_noent.rcf

On Linux, run the cell below:

In [ ]:
! ../02.6_Using_NAKSIN_4/NAKSIN_4/MoT-Voellmy-linux.2022-10-30 Test_case_MoT-Voellmy/Ryggfonn_1987-01-28_cst_noent.rcf

In the folder `Test_case_MoT-Voellmy`, there is a file `Ryggfonn.qgz`. Open it in QGIS to visualize your simulation results (there may be a warning that some files wee not found—press the button `Browse` to find them. You can add additional layers to view intermediate snapshots of the avalanche descent.

If you wish to explore the code further, modify the RCF, save it under a different name and run it in a new cell. For example, to find out how the coefficient *k* affects the speed and run-out, choose another constant value for *k* or replace the values for *µ* and *k* by the names of the files contained in the same folder. MoT-Voellmy also allows modeling snow entrainment along the path. To see this in action, indicate the file names `RGF_1987-01-28_b0.asc`, `RGF_1987-01-28_tauc.asc` in lines 13 and 14, and set the value of `Entrainment` to `TJEM`. To study the effect of the fracture depth, you would need to modify the raster file `RGF_1987-01-28_h0.asc`, e.g., by loading the file into an editor and replacing all instances of "1.0" by some other fracture depth.